In [1]:
import numpy as np
import pandas as pd
import spacy

In [2]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()


In [3]:
sent_df=pd.read_csv('../Downloads/archive (6)/train.csv',encoding='unicode_escape',usecols=['text','sentiment'])

In [4]:
from nltk.tokenize import word_tokenize
import re


In [5]:
sent_df.dropna(inplace=True)

In [6]:
sent_df

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
...,...,...
27476,wish we could come see u on Denver husband l...,negative
27477,I`ve wondered about rake to. The client has ...,negative
27478,Yay good for both of you. Enjoy the break - y...,positive
27479,But it was worth it ****.,positive


In [7]:
sent_df['sentiment']=le.fit_transform(sent_df['sentiment'])

In [8]:
sent_df

,text,sentiment
0,"I`d have responded, if I were going",1
1,Sooo SAD I will miss you here in San Diego!!!,0
2,my boss is bullying me...,0
3,what interview! leave me alone,0
4,"Sons of ****, why couldn`t they put them on t...",0
...,...,...
27476,wish we could come see u on Denver husband l...,0
27477,I`ve wondered about rake to. The client has ...,0
27478,Yay good for both of you. Enjoy the break - y...,2
27479,But it was worth it ****.,2


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(sent_df['text'],sent_df['sentiment'],test_size=0.2)

In [12]:
X_train=X_train.apply(url_rem)
X_test=X_test.apply(url_rem)

In [13]:
import string
exclude=string.punctuation
def punc_rem(text):
    return text.translate(str.maketrans('','',exclude)).strip()

In [14]:
X_train=X_train.apply(punc_rem)
X_test=X_test.apply(punc_rem)

In [15]:
X_train=X_train.str.lower()
X_test=X_test.str.lower()

In [16]:
import re
exc=re.compile(r'\d+')
def number_adj(txt):
    return re.sub(exc,'',txt)

In [17]:
X_train=X_train.apply(number_adj)
X_test=X_test.apply(number_adj)

In [18]:
from textblob import TextBlob


In [19]:
def word_correction(text):
    txt=TextBlob(text).correct()
    return txt.string

In [24]:
X_train.sample(100)

23162                         such a dissapointment  hhaha
22822                ugly what programmes do you have open
10939                                             nice one
5492                    i never was good at platform games
15062    have you managed to breathe since getting the ...
                               ...                        
12571    happy birthday  and  congrats  wish you were h...
16107    i wanted to see it until i saw that it was pg ...
26898    its am and both my girls are still up  im so t...
871                               oh no i hope its not bad
9208     oh hell i forgot the nonalcoholic cider for mo...
Name: text, Length: 100, dtype: object

In [25]:
def url_rem(txt):
     url_pattern = re.compile(r'https?://\S+|www\.\S+')
     return url_pattern.sub("", txt)

In [26]:
from autocorrect import Speller
spell=Speller(lang="en")

In [27]:
def spell_correct(text):
    sptext = []
    for doc in text:
        sptext.append(' '.join([spell(w).lower() for w in (doc)]))      
    return sptext

In [28]:
spell_correct(['hellloo'])

['h e l l l o o']

In [29]:
from nltk.tokenize import word_tokenize
doc_list=[]

In [30]:
def tokwnizer(doc_):
    doc_list.append(word_tokenize(doc_))

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()


In [32]:
x_tr=cv.fit_transform(X_train)
x_te=cv.transform(X_test)

In [33]:
x_tr.shape

(21984, 23296)

In [34]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()


In [35]:
gnb.fit(x_tr.toarray(),y_train)

GaussianNB()

In [36]:
y_pred=gnb.predict(x_te.toarray())

In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.3764556040756914

In [38]:
from sklearn.ensemble import RandomForestClassifier
rc=RandomForestClassifier(n_estimators=150,max_depth=70)

In [39]:
rc.fit(x_tr,y_train)

RandomForestClassifier(max_depth=70, n_estimators=150)

In [40]:
y_rc_pred=rc.predict(x_te)

In [41]:
accuracy_score(y_test,y_rc_pred)

0.6404657933042213

In [42]:
import matplotlib.pyplot as plt

In [46]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(max_iter=500)

In [47]:
lr.fit(x_tr,y_train)

LogisticRegression(max_iter=500)

In [48]:
y_lr_pred=lr.predict(x_te)

In [49]:
accuracy_score(y_test,y_lr_pred)

0.6844978165938864

In [51]:
x_te.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [52]:
X_test

18777    glad you liked it  hes done loads of arrangeme...
17157    yay my computer came today too bad its being h...
9067                                            is working
6227     a friend uses logic and i must admit i did lik...
10103    it is interesting that vs generates a setup sc...
                               ...                        
8100       ill visit ny til september no diggnation for me
14653    bye plurk out muna will be back when im done r...
5087     my baby is going to her first sleepover tonigh...
751      i want to experience snow  we dont have snow h...
22811            too easy happy mothers day to a great mom
Name: text, Length: 5496, dtype: object

In [89]:
tst=cv.transform(['He told us not to make so much noise. We were asked not to park in front of the house.'])
print(lr.predict(tst))

[0]
